## <font color='green'>2º Ejercicio</font>

Utilizaremos los siguientes csvs, disponibles en la carpeta *data*:
- empleados_tienda.csv
- facturacion_tienda.csv

Crea una nueva BBDD (comercio) y guarda los csvs en dos tablas (empleados, facturacion).

In [15]:
import sqlite3
import pandas as pd

In [16]:
conexion=sqlite3.connect('comercio')

In [17]:
df_empleados=pd.read_csv('../data/empleados_tienda.csv')
df_facturacion=pd.read_csv('../data/facturacion_tienda.csv')

In [18]:
df_empleados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_empleado      10 non-null     int64  
 1   nombre           10 non-null     object 
 2   apellido         10 non-null     object 
 3   id_tienda        10 non-null     int64  
 4   id_jefe_directo  8 non-null      float64
dtypes: float64(1), int64(2), object(2)
memory usage: 532.0+ bytes


In [19]:
df_empleados.head()

,id_empleado,nombre,apellido,id_tienda,id_jefe_directo
0,1,Ana,Garcia,1,NaN
1,2,Julia,Perez,1,1.0
2,3,Jon,Rodriguez,2,1.0
3,4,Iker,Etxeberria,2,3.0
4,5,Luisa,Garcia,3,1.0


In [20]:
df_facturacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id_establecimiento  5 non-null      int64 
 1   localidad           5 non-null      object
 2   facturacion         5 non-null      int64 
dtypes: int64(2), object(1)
memory usage: 252.0+ bytes


In [21]:
df_facturacion

,id_establecimiento,localidad,facturacion
0,1,Bilbao,1890000
1,2,Bilbao,783450
2,3,Barakaldo,372401
3,4,Leioa,280007
4,5,Barakaldo,462982


In [22]:
df_empleados.to_sql(name='empleados',con=conexion,index=False)

ValueError: Table 'empleados' already exists.

In [ ]:
df_facturacion.to_sql(name='facturacion',con=conexion,index=False)

5

In [ ]:
select_books = ''' 
SELECT * FROM sqlite_master WHERE type='table'
'''
pd.read_sql_query(select_books, con=conexion)

,type,name,tbl_name,rootpage,sql
0,table,empleados,empleados,2,"CREATE TABLE ""empleados"" (\n""id_empleado"" INTE..."
1,table,facturacion,facturacion,3,"CREATE TABLE ""facturacion"" (\n""id_establecimie..."


#### Realiza las siguientes consultas

¿Cuáles son las tiendas con más de 2 empleados?

In [ ]:
comando='''
SELECT id_tienda, COUNT(*) AS num_empleados
FROM empleados
GROUP BY id_tienda
HAVING COUNT(*) > 2
'''
pd.read_sql_query(comando, con=conexion)

,id_tienda,num_empleados
0,1,3
1,5,3


In [ ]:
comando='''
SELECT id_tienda, COUNT(*) AS num_empleados
FROM empleados
GROUP BY id_tienda
HAVING num_empleados>2
'''
pd.read_sql_query(comando, con=conexion)

,id_tienda,num_empleados
0,1,3
1,5,3


In [ ]:
comando='''
SELECT * FROM empleados
'''
pd.read_sql_query(comando, con=conexion)

,id_empleado,nombre,apellido,id_tienda,id_jefe_directo
0,1,Ana,Garcia,1,NaN
1,2,Julia,Perez,1,1.0
2,3,Jon,Rodriguez,2,1.0
3,4,Iker,Etxeberria,2,3.0
4,5,Luisa,Garcia,3,1.0
5,6,Fernando,Martin,4,NaN
6,7,Diana,Fernandez,5,6.0
7,8,Lorena,Uriarte,5,6.0
8,9,Sara,Fernandez,5,6.0
9,10,Inigo,Alzola,1,1.0


¿Cuántos de los empleados son jefes?

In [ ]:
comando=''' 
SELECT nombre, id_tienda  FROM empleados
WHERE id_tienda=1
'''

pd.read_sql_query(comando, con=conexion)

,nombre,id_tienda
0,Ana,1
1,Julia,1
2,Inigo,1


Ordena los tipos de jefes de mayor a menor según frecuencias.

In [ ]:
comando='''
SELECT nombre, CAST(id_jefe_directo AS INTEGER) AS jefe
FROM empleados
WHERE id_jefe_directo IS NOT NULL
ORDER BY jefe ASC
'''
#NAN NO LO COJE PONER IS NOR NULL
pd.read_sql_query(comando, con=conexion)

,nombre,jefe
0,Julia,1
1,Jon,1
2,Luisa,1
3,Inigo,1
4,Iker,3
5,Diana,6
6,Lorena,6
7,Sara,6


Ordena las tiendas de menor a mayor número de empleados.

In [ ]:
comando=''' SELECT id_tienda, COUNT(*) AS num_empleados
FROM empleados
GROUP BY id_tienda
ORDER BY num_empleados ASC
'''
pd.read_sql_query(comando, con=conexion)

,id_tienda,num_empleados
0,3,1
1,4,1
2,2,2
3,1,3
4,5,3


¿Cuáles son las tiendas que tienen el número máximo de empleados? (Puede haber más de una tienda que tenga el número máximo de empleados)

In [ ]:
comando=''' SELECT id_tienda, COUNT(*) AS num_empleados
FROM empleados
GROUP BY id_tienda
HAVING COUNT(*) = (
    SELECT COUNT(*) AS max_empleados
    FROM empleados
    GROUP BY id_tienda
    ORDER BY max_empleados DESC
    LIMIT 1
)
'''
pd.read_sql_query(comando, con=conexion)

,id_tienda,num_empleados
0,1,3
1,5,3


In [23]:
conexion.close()